In [19]:
import pandas as pd
import numpy as np

In [20]:
path = '../data/raw_data/women_boards_europe.xlsx'
files = ['total', 'detail']
sheets = ['women_boards_total', 'women_boards_detail']

In [21]:
labeledData = {k:pd.read_excel(path, v, dtype=str) for (k,v) in zip(files, sheets)} 

In [117]:
totalData = labeledData['total']
totalData.head()

,Largest listed companies,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Important information:,"Data on Presidents, members, and employee repr...",NaN,NaN,NaN,NaN,NaN
2,NaN,"Data on CEOs, executives and non-executives (i...",NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,For further information:,For methodological specifications refer to:,NaN,NaN,NaN,NaN,NaN


In [127]:
totalData = totalData[totalData.isnull().sum(axis=1) < 5]
totalData = totalData.iloc[1: , :]
totalData.reset_index(inplace=True)

In [128]:
columnNames = {'Largest listed companies': 'Country', 'Unnamed: 1' : 'Number_total', 
               'Unnamed: 2' : 'Number_women', 'Unnamed: 3' : 'Number_men', 
               'Unnamed: 4' : 'Percent_total', 'Unnamed: 5' : 'Percent_women',
               'Unnamed: 6' : 'Percent_men'} 

totalData.rename(columns = columnNames, inplace=True);

In [129]:
# This will convert all the columns to numeric except 'Country'
cols = totalData.columns.drop(['Country'])
totalData[cols] = totalData[cols].apply(pd.to_numeric, errors='coerce')

In [130]:
totalData.head()

,level_0,index,Country,Number_total,Number_women,Number_men,Percent_total,Percent_women,Percent_men
0,1,19,European Union - 28 countries (1993-2020),5896,1905,3991,100,32.3,67.7
1,2,20,Belgium,205,76,129,100,37.1,62.9
2,3,21,Bulgaria,59,10,49,100,16.9,83.1
3,4,22,Czechia,61,13,48,100,21.3,78.7
4,5,23,Denmark,235,91,144,100,38.7,61.3


In [198]:
detailData = labeledData['detail']
detailData.head()

,Year,_geo,Country,Value,UNIT,Unit,EGROUP,_EGROUP,Position,Position_init,sex,Gender,NACE,NACE_init
0,2012-B2,AT,Austria,0,Number of persons (headcount),NR,Largest listed companies,COMP,CEO (Chief Executive Officer),CEO,Women,W,"C, E & F (Mining & quarrying; electricity, gas...",CEF
1,2013-B1,AT,Austria,0,Number of persons (headcount),NR,Largest listed companies,COMP,CEO (Chief Executive Officer),CEO,Women,W,"C, E & F (Mining & quarrying; electricity, gas...",CEF
2,2013-B2,AT,Austria,0,Number of persons (headcount),NR,Largest listed companies,COMP,CEO (Chief Executive Officer),CEO,Women,W,"C, E & F (Mining & quarrying; electricity, gas...",CEF
3,2014-B1,AT,Austria,0,Number of persons (headcount),NR,Largest listed companies,COMP,CEO (Chief Executive Officer),CEO,Women,W,"C, E & F (Mining & quarrying; electricity, gas...",CEF
4,2014-B2,AT,Austria,0,Number of persons (headcount),NR,Largest listed companies,COMP,CEO (Chief Executive Officer),CEO,Women,W,"C, E & F (Mining & quarrying; electricity, gas...",CEF


In [199]:
detailData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 402180 entries, 0 to 402179
Data columns (total 14 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   Year           402180 non-null  object
 1   _geo           402180 non-null  object
 2   Country        402180 non-null  object
 3   Value          402180 non-null  object
 4   UNIT           402180 non-null  object
 5   Unit           402180 non-null  object
 6   EGROUP         402180 non-null  object
 7   _EGROUP        402180 non-null  object
 8   Position       402180 non-null  object
 9   Position_init  402180 non-null  object
 10  sex            402180 non-null  object
 11  Gender         402180 non-null  object
 12  NACE           402180 non-null  object
 13  NACE_init      402180 non-null  object
dtypes: object(14)
memory usage: 43.0+ MB


In [200]:
columnNames = {'time': 'Year', 'geo' : 'Country', '_UNIT' : 'Unit', 'value' : 'Value', 'POSITION' : 'Position',
               '_POSITION' : 'Position_init', '_sex' : 'Gender', 'NACE' : 'NACE', 
               '_NACE' : 'NACE_init'} 

keepColumns = ['Year', 'Country', 'Unit', 'Value', 'Position', 'Position_init', 'Gender', 'NACE', 'NACE_init']

detailData.rename(columns = columnNames, inplace=True)

detailData = detailData[keepColumns]

In [201]:
detailData['Year'] = detailData['Year'].apply(lambda x: x.split('-')[0]);

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [202]:
cols = detailData.columns.drop(['Country', 'Unit', 'Position', 'Position_init', 'Gender', 'NACE', 'NACE_init'])
detailData[cols] = detailData[cols].apply(pd.to_numeric, errors='coerce');

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [203]:
detailDataGroup = detailData.groupby(['Year','Country','Unit','Position','Position_init', 'Gender', 
                                 'NACE','NACE_init']
                               ).agg({'Value': 'sum'}
                               ).reset_index()

In [204]:
detailDataGroup['Number'] = np.where(detailDataGroup['Unit'] == 'NR', detailDataGroup['Value'], 0)
detailDataGroup['Percentage'] = np.where(detailDataGroup['Unit'] == 'PC', detailDataGroup['Value'], 0)

In [205]:
detailDataGroup.head()

,Year,Country,Unit,Position,Position_init,Gender,NACE,NACE_init,Value,Number,Percentage
0,2003,Austria,NR,Members,MEMB_BRD,M,"A Agriculture, hunting and forestry",A,6.0,6.0,0.0
1,2003,Austria,NR,Members,MEMB_BRD,M,"A, B, K-Q & Unknown - aggregate of sectors wit...",OTHER,117.0,117.0,0.0
2,2003,Austria,NR,Members,MEMB_BRD,M,All sectors,TOT,454.0,454.0,0.0
3,2003,Austria,NR,Members,MEMB_BRD,M,C Mining and quarrying,C,5.0,5.0,0.0
4,2003,Austria,NR,Members,MEMB_BRD,M,"C, E & F (Mining & quarrying; electricity, gas...",CEF,80.0,80.0,0.0


In [206]:
detailDataGroup = detailDataGroup.groupby(['Year','Country','Position','Position_init', 'Gender', 
                                 'NACE','NACE_init']
                               ).agg({'Number': 'sum', 'Percentage': 'sum'}
                               ).reset_index()

In [208]:
detailDataGroup.head()

,Year,Country,Position,Position_init,Gender,NACE,NACE_init,Number,Percentage
0,2003,Austria,Members,MEMB_BRD,M,"A Agriculture, hunting and forestry",A,6.0,85.7
1,2003,Austria,Members,MEMB_BRD,M,"A, B, K-Q & Unknown - aggregate of sectors wit...",OTHER,117.0,95.9
2,2003,Austria,Members,MEMB_BRD,M,All sectors,TOT,454.0,94.4
3,2003,Austria,Members,MEMB_BRD,M,C Mining and quarrying,C,5.0,100.0
4,2003,Austria,Members,MEMB_BRD,M,"C, E & F (Mining & quarrying; electricity, gas...",CEF,80.0,97.6


In [ ]:
fig, ax = plt.subplots(facecolor=c("grey1"), figsize=(15, 10))
ax.set_facecolor(c("grey1"))
colors = ['#c81200', '#c81200', '#eb3928', c('red'), '#d13a2d', '#df766b', '#e79289']
sns.set_palette(sns.color_palette(colors))
plt.stackplot(women_boards_europe_women_percentage_nace_1.year, women_boards_europe_women_percentage_nace_1.percentage, women_boards_europe_women_percentage_nace_2.percentage, women_boards_europe_women_percentage_nace_3.percentage, women_boards_europe_women_percentage_nace_4.percentage, women_boards_europe_women_percentage_nace_5.percentage, women_boards_europe_women_percentage_nace_6.percentage, women_boards_europe_women_percentage_nace_7.percentage, women_boards_europe_women_percentage_nace_8.percentage, women_boards_europe_women_percentage_nace_9.percentage, women_boards_europe_women_percentage_nace_10.percentage);
plt.legend(loc = 2, bbox_to_anchor = (1,1), prop={'size': 15}, facecolor=c('grey1'), edgecolor='white', labels=['Mining, electricity, gas and water', 'Manufacturing', 'Electricity, gas and water supply', 'Construction', 'Wholesale/retail, vehicle repair; hotels/restaurants; transport', 'Hotels and restaurants', 'Transport, storage and communication', 'Financial intermediation', 'Real estate, renting and business activities, consulting', 'aggregate of sectors with few large companies']);
plt.grid(visible=False)
plt.title("Evolution of the percentage of women in the boards of the largest companies in Europe by sectors", pad=30)